In [1]:
import pandas as pd
import numpy as np

In [54]:
manual = pd.read_csv("manual_cui_list.csv", header=None)
manual.columns = ["concept_code", "vocab", "CUI", "concept_name"]
neg_symptoms = ["Muscle Cramp", "Diarrhea", "Vomiting", "Nausea", "Fever"]
neg_binary = []
for concept_name in manual["concept_name"].values:
    print(concept_name)
    if concept_name in neg_symptoms:
        neg_binary.append(1)
    else:
        neg_binary.append(0)
manual["Negation?"] = neg_binary
# manual[manual["concept_name"].isin(neg_symptoms)]['Negative?'] = 1


# print(condition)
manual

Nausea
Muscle Cramp
Left upper quadrant pain
Diarrhea
Epigastric pain
Right upper quadrant pain
Shivering
Fever
Vomiting
Chronic abdominal pain
Recent weight loss
Decrease in appetite
Acute exacerbation of chronic abdominal pain
Distention


,concept_code,vocab,CUI,concept_name,Negation?
0,422587007,SNOMEDCT_US,C0027497,Nausea,1
1,55300003,SNOMEDCT_US,C0026821,Muscle Cramp,1
2,301715003,SNOMEDCT_US,C0238552,Left upper quadrant pain,0
3,62315008,SNOMEDCT_US,C0011991,Diarrhea,1
4,79922009,SNOMEDCT_US,C0232493,Epigastric pain,0
5,301717006,SNOMEDCT_US,C0235299,Right upper quadrant pain,0
6,43724002,SNOMEDCT_US,C0036973,Shivering,0
7,386661006,SNOMEDCT_US,C0015967,Fever,1
8,422400008,SNOMEDCT_US,C0042963,Vomiting,1
9,111985007,SNOMEDCT_US,C0232491,Chronic abdominal pain,0


In [11]:
import regex as re

with open("data/10498472.txt", "r") as f:
    auto = f.readlines()
auto

In [14]:
def extract_concept_data(text):
    # Compile the regular expressions for the fields we want to extract
    cui_pattern = re.compile(r"cui='(.*?)'")
    semtype_pattern = re.compile(r"semtypes='\[(.*?)\]'")
    preferred_name_pattern = re.compile(r"preferred_name='(.*?)'")
    trigger_pattern = re.compile(r"trigger='(.*?)'")

    # Search the text for matches
    cui_match = cui_pattern.search(text)
    semtype_match = semtype_pattern.search(text)
    preferred_name_match = preferred_name_pattern.search(text)
    trigger_match = trigger_pattern.search(text)

    # Extract the data using the search results
    cui = cui_match.group(1) if cui_match else None
    semtype = semtype_match.group(1) if semtype_match else None
    preferred_name = preferred_name_match.group(1) if preferred_name_match else None
    trigger = trigger_match.group(1) if trigger_match else None

    return cui, semtype, preferred_name, trigger

In [16]:
with open("data/10498472.txt", "r") as f:
    for line in f:
        cui, semtype, preferred_name, trigger = extract_concept_data(line)
        # print(cui, semtype, preferred_name, trigger)
        if cui in manual["CUI"].values:
            print(
                cui,
                semtype,
                preferred_name,
                trigger,
                manual[manual["CUI"] == cui]["concept_name"],
            )

C0232491 sosy Chronic abdominal pain ["CHRONIC ABDOMINAL PAIN"-tx-2-"chronic abdominal pain"-noun-0,"CHRONIC ABDOMINAL PAIN"-tx-1-"chronic abdominal pain"-noun-0] 9    Chronic abdominal pain
Name: concept_name, dtype: object
C0232493 sosy Epigastric pain ["Epigastric pain"-tx-5-"epigastric pain"-noun-0] 4    Epigastric pain
Name: concept_name, dtype: object
C0027497 sosy Nausea ["NAUSEA"-tx-4-"nausea"-noun-0] 0    Nausea
Name: concept_name, dtype: object


In [23]:
with open("data/10498472.txt", "r") as f:
    for line in f:
        cui, semtype, preferred_name, trigger = extract_concept_data(line)
        preferred_name = preferred_name.replace(",", ";")
        trigger = trigger.replace(",", ";")

        print(cui, semtype, preferred_name, trigger)
        with open("mm_cui_list.csv", "a") as f:
            f.write(cui + "," + semtype + "," + preferred_name + "," + trigger + "\n")

C0030193 sosy Pain ["PAIN"-tx-6-"pain"-noun-0;"PAIN"-tx-5-"pain"-noun-0;"PAIN"-tx-2-"pain"-noun-0]
C0232491 sosy Chronic abdominal pain ["CHRONIC ABDOMINAL PAIN"-tx-2-"chronic abdominal pain"-noun-0;"CHRONIC ABDOMINAL PAIN"-tx-1-"chronic abdominal pain"-noun-0]
C0232493 sosy Epigastric pain ["Epigastric pain"-tx-5-"epigastric pain"-noun-0]
C0853946 sosy pain aggravated ["pain worsening"-tx-16-"pain worsening"-noun-0]
C0740577 sosy Acute abdominal pain ["Acute abdominal pain"-tx-1-"acute abdominal pain"-noun-0]
C0474496 sosy Diarrhea and vomiting; symptom ["vomiting diarrhoea"-tx-4-"vomiting diarrhea"-noun-1]
C0027497 sosy Nausea ["NAUSEA"-tx-4-"nausea"-noun-0]
C0085594 sosy Fever with chills ["FEVER CHILLS"-tx-4-"fevers chills"-noun-1]
C1446787 sosy Cramping sensation quality ["Crampy"-tx-3-"crampy"-adj-1]
C1112705 dsyn Nuclear non-senile cataract ["NS"-tx-15-"NS"-noun-0]
C0577573 sosy Mass of body region ["Mass"-tx-14-"mass"-noun-0]
C0268529 dsyn Proline dehydrogenase deficiency ["Hyp

In [41]:
mm = pd.read_csv("mm_cui_list.csv", header=None)
mm.columns = ["CUI", "semtype", "preferred_name", "trigger"]

In [42]:
import re

# Regex pattern to match the negation flag after '-adj-' or '-noun-' and before ']'
negation_flag_pattern = r"-(?:adj|noun)-"  # )\d+-"\w+"-(\d)')


def extract_negation_flags(trigger_list):
    negation_flags = []
    for trigger in trigger_list:
        # Find all matches of the negation flag pattern in the trigger string
        matches = re.search(f"{negation_flag_pattern}(\w+)", trigger)
        # Each match is a tuple, and the flag can be in either position depending on the match
        if matches:
            negation_flags.extend(matches.groups())

    return [int(i) for i in negation_flags]
    # negation_flags.extend(matches)
    # return negation_flags


# Example list of triggers including negation flags
triggers = [
    '["PAIN"-tx-6-"pain"-noun-0]',
    '["CHRONIC ABDOMINAL PAIN"-noun-1;"chronic abdominal pain"-noun-0]',
    '["Epigastric pain"-tx-5-"epigastric pain"-noun-1]',
    # ... more triggers ...
]

# Extract negation flags
negation_flags = extract_negation_flags(mm.trigger.values)
print(negation_flags)

[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]


In [55]:
mm["Negation?"] = negation_flags
# reorder columns
mm = mm[
    [
        "CUI",
        "semtype",
        "preferred_name",
        "Negation?",
        "trigger",
    ]
]
mm[mm.semtype == "sosy"]

,CUI,semtype,preferred_name,Negation?,trigger
0,C0030193,sosy,Pain,0,"[""PAIN""-tx-6-""pain""-noun-0;""PAIN""-tx-5-""pain""-..."
1,C0232491,sosy,Chronic abdominal pain,0,"[""CHRONIC ABDOMINAL PAIN""-tx-2-""chronic abdomi..."
2,C0232493,sosy,Epigastric pain,0,"[""Epigastric pain""-tx-5-""epigastric pain""-noun-0]"
3,C0853946,sosy,pain aggravated,0,"[""pain worsening""-tx-16-""pain worsening""-noun-0]"
4,C0740577,sosy,Acute abdominal pain,0,"[""Acute abdominal pain""-tx-1-""acute abdominal ..."
5,C0474496,sosy,Diarrhea and vomiting; symptom,1,"[""vomiting diarrhoea""-tx-4-""vomiting diarrhea""..."
6,C0027497,sosy,Nausea,0,"[""NAUSEA""-tx-4-""nausea""-noun-0]"
7,C0085594,sosy,Fever with chills,1,"[""FEVER CHILLS""-tx-4-""fevers chills""-noun-1]"
8,C1446787,sosy,Cramping sensation quality,1,"[""Crampy""-tx-3-""crampy""-adj-1]"
10,C0577573,sosy,Mass of body region,0,"[""Mass""-tx-14-""mass""-noun-0]"
